# Weekly Report for Hospital Data

In [16]:
# Imports
import sys
import pandas as pd
import psycopg
from loadinghelper import check_numeric_na

from credentials import DB_PASSWORD, DB_USER

In [17]:
records["Total Adult Hospital Beds Available"].apply(check_numeric_na)

InvalidOperation: [<class 'decimal.InvalidOperation'>]

In [2]:
# Connect to DB
conn = psycopg.connect(
    host="sculptor.stat.cmu.edu", dbname=DB_USER,
    user=DB_USER, password=DB_PASSWORD
)
cur = conn.cursor()

In [3]:
# Summary I & II
with conn.transaction():
    cur.execute(
        "SELECT report_date AS week, COUNT(*) AS records_loaded, "
        "SUM(total_adult_hospital_beds), SUM(total_pediatric_hospital_beds), "
        "SUM(total_adult_hospital_beds_occupied), "
        "SUM(total_pediatric_hospital_beds_occupied), "
        "SUM(inpatient_beds_occupied_covid) "
        "FROM facility_reports "
        "GROUP BY report_date "
        "ORDER BY week DESC;"
    )

records = pd.DataFrame(
    cur.fetchall(), columns=["Week", "Records Loaded", "Total Adult Hospital Beds Available", 
                             "Total Pediatric Hospital Beds Available", 
                             "Total Adult Hospital Beds In-Use", "Total Pediatric Hospital Beds In-Use",
                             "Total Hospital Beds Used by COVID cases"]
)

In [4]:
# I
records[["Week", "Records Loaded"]]

,Week,Records Loaded
0,2022-10-21,4985
1,2022-10-14,4995
2,2022-10-07,4996
3,2022-09-30,4997
4,2022-09-23,4996


In [5]:
# II
records

,Week,Records Loaded,Total Adult Hospital Beds Available,Total Pediatric Hospital Beds Available,Total Adult Hospital Beds In-Use,Total Pediatric Hospital Beds In-Use,Total Hospital Beds Used by COVID cases
0,2022-10-21,4985,NaN,NaN,NaN,NaN,NaN
1,2022-10-14,4995,NaN,NaN,NaN,NaN,NaN
2,2022-10-07,4996,NaN,NaN,NaN,NaN,NaN
3,2022-09-30,4997,NaN,NaN,NaN,NaN,NaN
4,2022-09-23,4996,NaN,NaN,NaN,NaN,NaN
